In [4]:
#---------------------------------模型引入---------------------------------
import csv, os, time, mindspore
import numpy as np
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
from mindspore import nn, context, dataset, Tensor
from mindspore.train.callback import TimeMonitor, LossMonitor
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
# 设定运行模式为静态图模式，并且运行设备为昇腾芯片
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend") 

#---------------------------------变量定义---------------------------------
cfg = edict({
    'data_size': 150,
    'train_size': 120, #训练集大小
    'test_size': 30 , #测试集大小
    'feature_number': 4, #输入特征数
    'num_class': 3, #分类类别
    'batch_size': 30, #批次大小
    'data_dir': 'iris.data', # 数据集路径
    'save_checkpoint_steps': 1, #多少步保存一次模型
    'keep_checkpoint_max': 1, #最多保存多少个模型
    'out_dir_no_opt': 'no_opt', #保存模型路径，无优化器模型
    'out_dir_sgd': './model_iris/sgd', #保存模型路径,SGD优化器模型
    'out_dir_momentum': './model_iris/momentum', #保存模型路径，momentum模型
    'out_dir_RMSProp': './model_iris/RMSProp',
    'out_dir_adam': './model_iris/adam', #保存模型路径，adam优化器模型
    'class_names': ['setosa', 'versicolor', 'virginica'] #分类标签
})

#---------------------------------数据读取---------------------------------
#鸢尾花数据集，本数据集共有150个带标签的数据
with open(cfg.data_dir) as csv_file:
    data = list(csv.reader(csv_file, delimiter=','))
label_map = {'setosa': 0,'versicolor': 1,'virginica':2 }
#分别获取数据中的特征值X和标签值Y
X = np.array([[float(x) for x in s[:-1]] for s in data[:cfg.data_size]],np.float32)
Y = np.array([label_map[s[-1]] for s in data[:cfg.data_size]], np.int32)
# 将数据集分为训练集120条，测试集30条。
train_idx = np.random.choice(cfg.data_size, cfg.train_size, replace=False)
test_idx = np.array(list(set(range(cfg.data_size)) - set(train_idx)))
X_train, Y_train = X[train_idx], Y[train_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

#---------------------------------数据预处理---------------------------------

# 构造训练集可迭代式对象
class IterDatasetGenerator_train:
    def __init__(self):
        self.__index = 0
        self.__data = X_train
        self.__label = Y_train

    def __next__(self):
        if self.__index >= len(self.__data):
            raise StopIteration
        else:
            item = (self.__data[self.__index], self.__label[self.__index])
            self.__index += 1
            return item

    def __iter__(self):
        self.__index = 0
        return self

    def __len__(self):
        return len(self.__data)
    
# 构造测试集可迭代式对象
class IterDatasetGenerator_test:
    def __init__(self):
        self.__index = 0
        self.__data = X_test
        self.__label = Y_test

    def __next__(self):
        if self.__index >= len(self.__data):
            raise StopIteration
        else:
            item = (self.__data[self.__index], self.__label[self.__index])
            self.__index += 1
            return item

    def __iter__(self):
        self.__index = 0
        return self

    def __len__(self):
        return len(self.__data)
# dataset 生成器
dataset_generator_train = IterDatasetGenerator_train()
dataset_generator_test  = IterDatasetGenerator_test()

def pre_process_data(X_train, Y_train):
    #生成训练集和测试集
    ds_train = dataset.GeneratorDataset(dataset_generator_train, ['x', 'y'])
    ds_test  = dataset.GeneratorDataset(dataset_generator_test, ['x', 'y'])
    ds_train = ds_train.shuffle(buffer_size=cfg.train_size)
    ds_test  = ds_test.shuffle(buffer_size=cfg.test_size)
    ds_train = ds_train.batch(batch_size=cfg.batch_size, drop_remainder=True)
    '''
    :param batch_size:  表示要在单个批次中组合的此数据集的连续元素数
    :param drop_remainder: 示如果最后一批元素少于批次大小，是否应删除最后一批元素，默认为False。
    :param num_parallel_calls: 表示异步并行计算的批数。如果未指定，将按顺序计算批次
    :param deterministic: 被指定时，如果指定了此布尔值（True或False），则它控制转换生成元素的顺序。
    '''
    ds_test  = ds_test.batch (batch_size=cfg.test_size , drop_remainder=True)
    return ds_train, ds_test

#---------------------------------构建全连接神经网络---------------------------------
# 全连接网络(一层隐藏层)
class FC(nn.Cell):
    # 神经网络的各层需要预先在__init__方法中定义
    def __init__(self):
        super().__init__()
        self.num_class = cfg.num_class
        self.fc1 = nn.Dense(cfg.feature_number, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Dense(10, self.num_class)
    # 在construct中完成神经网络的前向计算
    def construct(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
network = FC()

#---------------------------------模型训练---------------------------------
def train(net_opt, ds_train, prefix, directory, epoch_size):
    #定义网络损失函数
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    #定义模型
    model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={"acc"})
    '''
    Model中包含以下参数
    network--一般情况下为前向网络，输入数据和标签，输出预测值
    loss_fn--所使用的损失函数
    optimizer--所使用的优化器
    metrics--进行模型评估时使用的评价指标，在不需要模型评估时使用默认值
    eval_network--模型评估所使用的网络，在部分简单场景下不需要指定
    wval-indexes--用于指示评估网络输出的含义
    amp_level--用于指定混合精度级别
    kwargs--可配置溢出检测和混合精度策略
    Model提供以下接口
    train--用于在训练集上进行模型训练
    eval--用于在验证集上进行模型评估
    predict--用于对输入的一组数据进行推理，输出预测结果
    '''
    #定义损失值指标
    loss_cb = LossMonitor(per_print_times=int(cfg.train_size / cfg.batch_size))
    #设置checkpoint
    #用户可以根据具体需求对CheckPoint策略进行配置
    config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,
                                 keep_checkpoint_max=cfg.keep_checkpoint_max)
    '''
    上述代码中，首先需要初始化一个CheckpointConfig类对象，用来设置保存策略
    save_checkpoint_steps--每隔多少个step保存一次。
    keep_checkpoint_max----最多保留CheckPoint文件的数量。
    prefix-----------------生成CheckPoint文件的前缀名。
    directory--------------存放文件的目录。
    '''
    '''
    创建一个ModelCheckpoint对象把它传递给model.train方法
    就可以在训练过程中使用CheckPoint功能了
    '''
    ckpoint_cb = ModelCheckpoint(prefix=prefix, directory=directory, config=config_ck)
    print("============== Starting Training ==============")
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, loss_cb], dataset_sink_mode=False)
    print("============== Train Finished ==============")
    return model

#---------------------------------评估预测---------------------------------
def eval_predict(model, ds_test):
    # 使用测试集评估模型，打印总体准确率
    metric = model.eval(ds_test)
    print(metric)
    # 预测
    test_ = ds_test.create_dict_iterator().__next__()
    test = Tensor(test_['x'], mindspore.float32)
    predictions = model.predict(test)
    predictions = predictions.asnumpy()
    true_label = test_['y'].asnumpy()
    for i in range(15):
        p_np = predictions[i, :]
        pre_label = np.argmax(p_np)
        print('第' + str(i) + '个sample预测结果：', cfg.class_names[pre_label], '   真实结果：', cfg.class_names[true_label[i]])


#---------------------------------模型预测---------------------------------
def run_no_opt():
    ds_train, ds_test = pre_process_data(X_train, Y_train)
    print('---------------------------------NO_OPT---------------------------------')
    print("============== Pre Process Finished ==============")
    model = train(None, ds_train, "checkpoint_no_opt.ckpt", cfg.out_dir_no_opt, 10)
    eval_predict(model, ds_test)
    print("============== NO_OPT Evaluate Finished ==============")

def run_SGD():
    ds_train, ds_test = pre_process_data(X_train, Y_train)
    print('---------------------------------SGD---------------------------------')
    print("============== Pre Process Finished ==============")
    optimizer = nn.SGD(params=network.trainable_params(), learning_rate=0.01)
    model = train(optimizer, ds_train, "checkpoint_sgd", cfg.out_dir_sgd, 200)
    eval_predict(model, ds_test)
    print("============== SGD Evaluate Finished ==============")

def run_Momentum():
    ds_train, ds_test = pre_process_data(X_train, Y_train)
    print('---------------------------------Momentum---------------------------------')
    print("============== Pre Process Finished ==============")
    optimizer = nn.Momentum(params=network.trainable_params(), learning_rate=0.01, momentum=0.9)
    model = train(optimizer, ds_train, "checkpoint_momentum", cfg.out_dir_momentum, 100)
    eval_predict(model, ds_test)
    print("============== Momentum Evaluate Finished ==============")

def run_RMSProp():
    ds_train, ds_test = pre_process_data(X_train, Y_train)
    print('---------------------------------RMSProp---------------------------------')
    print("============== Pre Process Finished ==============")
    optimizer = nn.RMSProp(params=network.trainable_params(), learning_rate=0.01, weight_decay=0.0)
    model = train(optimizer, ds_train, "checkpoint_RMSProp", cfg.out_dir_RMSProp, 50)
    eval_predict(model, ds_test)
    print("============== RMSProp Evaluate Finished ==============")
    
def run_Adam():
    ds_train, ds_test = pre_process_data(X_train, Y_train)
    print('---------------------------------Adam---------------------------------')
    print("============== Pre Process Finished ==============")
    optimizer = nn.Adam(params=network.trainable_params(), learning_rate=0.1, weight_decay=0.0)
    model = train(optimizer, ds_train, "checkpoint_adam", cfg.out_dir_adam, 15)
    eval_predict(model, ds_test)
    print("============== Adam Evaluate Finished ==============")
#---------------------------------运行---------------------------------
run_no_opt()
# run_SGD()
# run_Momentum()
# run_RMSProp()
# run_Adam()

---------------------------------NO_OPT---------------------------------
============== Pre Process Finished ==============
============== Starting Training ==============
epoch: 1 step: 4, loss is 1.0989527
epoch: 2 step: 4, loss is 1.0986707
epoch: 3 step: 4, loss is 1.0980687
epoch: 4 step: 4, loss is 1.098439
epoch: 5 step: 4, loss is 1.0991024
epoch: 6 step: 4, loss is 1.098732
epoch: 7 step: 4, loss is 1.0991039
epoch: 8 step: 4, loss is 1.0979238
epoch: 9 step: 4, loss is 1.0987964
epoch: 10 step: 4, loss is 1.0988059
============== Train Finished ==============
{'acc': 0.43333333333333335}
第0个sample预测结果： versicolor    真实结果： setosa
第1个sample预测结果： versicolor    真实结果： virginica
第2个sample预测结果： versicolor    真实结果： setosa
第3个sample预测结果： versicolor    真实结果： versicolor
第4个sample预测结果： versicolor    真实结果： versicolor
第5个sample预测结果： versicolor    真实结果： versicolor
第6个sample预测结果： versicolor    真实结果： versicolor
第7个sample预测结果： versicolor    真实结果： virginica
第8个sample预测结果： versicolor    真实结果： ve